# Basic Weather Agent with MCP Support (SSE Transport)

* Collaborator: Roberto Rodriguez @Cyb3rWard0g

## Install Required Libraries
Before starting, ensure the required libraries are installed:

In [ ]:
!pip install dapr-agents python-dotenv mcp starlette

## Import Environment Variables

In [1]:
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.

True

## Enable Logging

In [2]:
import logging

logging.basicConfig(level=logging.INFO)

## Connect to MCP Server and Get Tools

In [ ]:
# Initialize MCP SSE Server
# uv run servers/weather_server.py --server_type sse

In [3]:
from mcp.client.sse import sse_client
from mcp import ClientSession

from dapr_agents import Agent, AgentTool

# Connect to the server using SSE
async with sse_client("http://localhost:8000/sse") as (read, write):
    async with ClientSession(read, write) as session:
        # Initialize the session
        await session.initialize()
        
        # Get the agent tools from the MCP session
        agent_tools = await AgentTool.from_mcp_session(session)
        
        # Create an agent with the tools
        agent = Agent(
            name="Rob",
            role="Weather Assistant",
            tools=agent_tools
        )

        # Run the agent with a sample query
        await agent.run("What is the weather in Virginia?")

/Users/wardog/Documents/GitHub/dapr-agents/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:mcp.client.sse:Connecting to SSE endpoint: http://localhost:8000/sse
INFO:httpx:HTTP Request: GET http://localhost:8000/sse "HTTP/1.1 200 OK"
INFO:mcp.client.sse:Received endpoint URL: http://localhost:8000/messages/?session_id=eb9d73eedeec4c78a9ab4746112b1e80
INFO:mcp.client.sse:Starting post writer with endpoint URL: http://localhost:8000/messages/?session_id=eb9d73eedeec4c78a9ab4746112b1e80
INFO:httpx:HTTP Request: POST http://localhost:8000/messages/?session_id=eb9d73eedeec4c78a9ab4746112b1e80 "HTTP/1.1 202 Accepted"
INFO:httpx:HTTP Request: POST http://localhost:8000/messages/?session_id=eb9d73eedeec4c78a9ab4746112b1e80 "HTTP/1.1 202 Accepted"
INFO:httpx:HTTP Request: POST http://localhost:8000/mes

user:
What is the weather in Virginia?

--------------------------------------------------------------------------------



INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:dapr_agents.llm.openai.chat:Chat completion retrieved successfully.
INFO:dapr_agents.agents.agent.agent:Executing GetWeather with arguments {'location': 'Virginia'}
INFO:dapr_agents.tool.executor:Running tool (auto): GetWeather
INFO:httpx:HTTP Request: POST http://localhost:8000/messages/?session_id=eb9d73eedeec4c78a9ab4746112b1e80 "HTTP/1.1 202 Accepted"
INFO:dapr_agents.agents.agent.agent:Iteration 2/10 started.
INFO:dapr_agents.llm.utils.request:Tools are available in the request.
INFO:dapr_agents.llm.openai.chat:Invoking ChatCompletion API.


assistant:
{'content': None, 'role': 'assistant', 'tool_calls': [{'id': 'call_EA0qfyPxmFx69NG5iuGi6EhE', 'type': 'function', 'function': {'name': 'GetWeather', 'arguments': '{"location":"Virginia"}'}}]}

--------------------------------------------------------------------------------

GetWeather(tool) (Id: call_EA0qfyPxmFx69NG5iuGi6EhE):
Virginia: 76F.

--------------------------------------------------------------------------------



INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:dapr_agents.llm.openai.chat:Chat completion retrieved successfully.


assistant:
{'content': 'The current temperature in Virginia is 76°F.', 'role': 'assistant'}

--------------------------------------------------------------------------------



## Connect to SSE MCP Servers

In [ ]:
# Initialize MCP SSE Server
# uv run servers/weather_server.py --server_type sse

In [5]:
# Define your MCP server configs
mcp_servers = {
    "weather": {
        "transport": "sse",
        "url": "http://localhost:8000/sse"
    },
}

In [6]:
# Note: The server script (serverA.py) should be running for this to work.
from dapr_agents.tool.mcp.client import MCPClient

# Create an instance of the MCPClient to interact with the MCP servers
#client = MCPClient(persistent_connections=True)
client = MCPClient()

In [7]:
# Connect to the MCP servers and get tools and prompts
await client.connect_from_config(mcp_servers)

INFO:mcp.client.sse:Connecting to SSE endpoint: http://localhost:8000/sse
INFO:httpx:HTTP Request: GET http://localhost:8000/sse "HTTP/1.1 200 OK"
INFO:mcp.client.sse:Received endpoint URL: http://localhost:8000/messages/?session_id=45afbd0c23b74c3abe0c2533a581a210
INFO:mcp.client.sse:Starting post writer with endpoint URL: http://localhost:8000/messages/?session_id=45afbd0c23b74c3abe0c2533a581a210
INFO:httpx:HTTP Request: POST http://localhost:8000/messages/?session_id=45afbd0c23b74c3abe0c2533a581a210 "HTTP/1.1 202 Accepted"
INFO:httpx:HTTP Request: POST http://localhost:8000/messages/?session_id=45afbd0c23b74c3abe0c2533a581a210 "HTTP/1.1 202 Accepted"
INFO:httpx:HTTP Request: POST http://localhost:8000/messages/?session_id=45afbd0c23b74c3abe0c2533a581a210 "HTTP/1.1 202 Accepted"
INFO:dapr_agents.tool.mcp.client:Loaded 2 tools from server 'weather'
INFO:httpx:HTTP Request: POST http://localhost:8000/messages/?session_id=45afbd0c23b74c3abe0c2533a581a210 "HTTP/1.1 202 Accepted"
INFO:dap

In [8]:
# Get all tools available
tools = client.get_all_tools()
print("🔧 Tools:", [t.name for t in tools])

🔧 Tools: ['WeatherGetWeather', 'WeatherJump']


## Initialize Agent

In [9]:
from dapr_agents import Agent

agent = Agent(
    name="Rob",
    role= "Weather Assistant",
    tools=tools
)

INFO:dapr_agents.llm.openai.client.base:Initializing OpenAI client...
INFO:dapr_agents.tool.executor:Tool registered: WeatherGetWeather
INFO:dapr_agents.tool.executor:Tool registered: WeatherJump
INFO:dapr_agents.tool.executor:Tool Executor initialized with 2 tool(s).
INFO:dapr_agents.agents.base:Constructing system_prompt from agent attributes.
INFO:dapr_agents.agents.base:Using system_prompt to create the prompt template.
INFO:dapr_agents.agents.base:Pre-filled prompt template with attributes: ['name', 'role', 'goal']


## Run Agent

In [10]:
await agent.run("What is the weather in New York?")

INFO:dapr_agents.agents.agent.agent:Iteration 1/10 started.
INFO:dapr_agents.llm.utils.request:Tools are available in the request.
INFO:dapr_agents.llm.openai.chat:Invoking ChatCompletion API.


user:
What is the weather in New York?

--------------------------------------------------------------------------------



INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:dapr_agents.llm.openai.chat:Chat completion retrieved successfully.
INFO:dapr_agents.agents.agent.agent:Executing WeatherGetWeather with arguments {'location': 'New York'}
INFO:dapr_agents.tool.executor:Running tool (auto): WeatherGetWeather
INFO:mcp.client.sse:Connecting to SSE endpoint: http://localhost:8000/sse
INFO:httpx:HTTP Request: GET http://localhost:8000/sse "HTTP/1.1 200 OK"
INFO:mcp.client.sse:Received endpoint URL: http://localhost:8000/messages/?session_id=ae3125738c4c4835a1365a706b5cc525
INFO:mcp.client.sse:Starting post writer with endpoint URL: http://localhost:8000/messages/?session_id=ae3125738c4c4835a1365a706b5cc525
INFO:httpx:HTTP Request: POST http://localhost:8000/messages/?session_id=ae3125738c4c4835a1365a706b5cc525 "HTTP/1.1 202 Accepted"
INFO:httpx:HTTP Request: POST http://localhost:8000/messages/?session_id=ae3125738c4c4835a1365a706b5cc525 "HTTP/1.1 202 Accepted"


assistant:
{'content': None, 'role': 'assistant', 'tool_calls': [{'id': 'call_hv7dcLtjtk9JEjgwrrguBP3C', 'type': 'function', 'function': {'name': 'WeatherGetWeather', 'arguments': '{"location":"New York"}'}}]}

--------------------------------------------------------------------------------

WeatherGetWeather(tool) (Id: call_hv7dcLtjtk9JEjgwrrguBP3C):
New York: 78F.

--------------------------------------------------------------------------------



INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:dapr_agents.llm.openai.chat:Chat completion retrieved successfully.


assistant:
{'content': 'The current temperature in New York is 78°F.', 'role': 'assistant'}

--------------------------------------------------------------------------------



'The current temperature in New York is 78°F.'